In [11]:
import pandas as pd
df = pd.read_csv('./../data/train.csv')
train_mean=0
train_std=0
print(df.head())
len(df.index)

   page_likes  page_checkin  daily_crowd  page_category   F1          F2  \
0      400487             0        57833             13  414   26.635492   
1     2667410           141       111855             18  793  185.317073   
2     2411555             0        61797              9  560  103.860465   
3        1013             0            1             85   16    2.224299   
4      367390             0         2678            100  110   11.828175   

      F3          F4  F5   F6   ...     c4  c5  base_time  post_length  \
0   15.0   44.941598   0  300   ...     43  43         24          100   
1  157.0  164.739179   0  419   ...    216 -29         68          118   
2   78.0  107.011971   0  325   ...      0   0          0          119   
3    1.0    3.489124   0    7   ...      1   1          1            0   
4    6.0   15.430200   0   96   ...      9   9         19           53   

   share_count  promotion  h_target   post_day  basetime_day  target  
0           20          0  

32000

In [12]:
def preprocessData(df):
    df['post_day'] = df['post_day'].factorize(sort=True)[0]
    df['basetime_day'] = df['basetime_day'].factorize(sort=True)[0]
    df = df.sample(frac=1).reset_index(drop=True)
    df=df.head(2000)
#     print(df)
    df_norm=df.drop(labels='target', axis=1)
    print(df_norm.mean())
    print(df_norm.std())
    df_norm=(df_norm-df_norm.mean())/df_norm.std()
    global train_mean
    global train_std
    train_mean=df_norm.mean()
    train_std=df_norm.std()
#     df_norm=(df_norm-df_norm.min())/(df_norm.max()-df_norm.min())
    df_norm.fillna(0,inplace=True)
    print(df_norm.head(5))
    df_norm=pd.concat([df_norm, df['target']], axis=1)
#     print(df_norm)
#     df_norm=df_norm[['page_likes','daily_crowd','target']]
    print(df_norm)
    return df_norm

In [13]:
val_percent=0.1

train_data=preprocessData(df)
val_size=int(val_percent*len(train_data.index))
val_data=train_data.tail(val_size)
train_data=train_data.head(len(train_data.index)-val_size)

page_likes       1.562306e+06
page_checkin     4.534792e+03
daily_crowd      4.891166e+04
page_category    2.444850e+01
F1               4.864565e+02
F2               5.659028e+01
F3               3.605875e+01
F4               6.867570e+01
F5               9.200000e-02
F6               3.814870e+02
F7               2.209677e+01
F8               7.237000e+00
c1               6.185450e+01
c2               2.146650e+01
c3               2.263100e+01
c4               5.803050e+01
c5              -1.164500e+00
base_time        3.533450e+01
post_length      1.594690e+02
share_count      1.372305e+02
promotion        0.000000e+00
h_target         2.379850e+01
post_day         3.086000e+00
basetime_day     3.079500e+00
dtype: float64
page_likes       1.130892e+07
page_checkin     2.035203e+04
daily_crowd      1.335142e+05
page_category    1.991760e+01
F1               5.446901e+02
F2               8.925000e+01
F3               7.323778e+01
F4               8.337938e+01
F5               1.872509

In [14]:
def onehotencoding(y):
    y=y.astype(int)
#     print(y.shape)
    max_y=max(y)
#     max_y=3
    encode_mat=np.zeros((y.shape[0],max_y))
    rows=np.arange(y.shape[0])
    #array([0, 1, 2, 3, 4])
    encode_mat[rows,y-1]=1
    return encode_mat

In [15]:
import numpy as np
class nn:
    #nn for classifier works for only one layer
    def __init__(self, no_of_inputs, no_of_outputs, HUs,no_of_hidden_layers=1, activation=["sigmoid","sigmoid"],regu=0, dropout=0, weights_seed=0):
        
        
        
        self.no_of_inputs=no_of_inputs
        self.no_of_hidden_layers=no_of_hidden_layers
        self.no_of_outputs=no_of_outputs
        self.HUs=HUs #hidden units array
        self.act=activation
        self.dropout=dropout
        self.weights_seed=weights_seed
        self.regu=regu
        #sanity checks
        if(len(self.HUs) != self.no_of_hidden_layers):
            print("error mismatch hidden units and layers")
        if((self.no_of_hidden_layers+1)!=len(activation)):
            print("error mismatch activations and layers")
        self.initialize_weights()
    
    def initialize_weights(self):
        
        np.random.seed(self.weights_seed)
        self.weights=[]
        weights0=0.01 * np.random.randn(self.no_of_inputs+1,self.HUs[0])
        self.weights.append(weights0);
        # to add multiple layers
        for i in range(1,self.no_of_hidden_layers):
            weightsh=0.01 * np.random.randn(self.HUs[i-1]+1,self.HUs[i])
            self.weights.append(np.copy(weightsh));
              
            
        weightsl=0.01 * np.random.randn(self.HUs[-1]+1,self.no_of_outputs)
        self.weights.append(weightsl);
    
    def get_weights(self):
        return self.weights
    
    def sigmoid_forward(self,z):
#         print("sig forward")
        return 1/(1+np.exp(-z))
        
    def sigmoid_backward(self,z):
#         print("sig bckward")
        return z*(1-z)
    
    def tanh_forward(self,z):
#         print("tanh forwad")
        return np.tanh(z) 
    
    def tanh_backward(self,z):
#         print("tanh backward")
        return 1-z**2
    
    #not working relu
    def relu_forward(self,z):
#         print("relu forwad")
        return z * (z > 0) 
    
    def relu_backward(self,z):
#         print("relu backward")
        
        return 1 * (z > 0)
    
    def softplus_forward(self,z):
        return np.log(1+np.exp(z)) 
    
    def softplus_backward(self,z): 
        return 1-1/np.exp(z)
    
    def activation_forward(self,z,act_index):
        if(self.act[act_index]=="sigmoid"):
            return self.sigmoid_forward(z)
        elif (self.act[act_index]=="softplus"):
            return self.softplus_forward(z)
        elif (self.act[act_index]=="tanh"):
            return self.tanh_forward(z)
        elif (self.act[act_index]=="relu"):
            return self.relu_forward(z)
        else:
            print("error invalid activation")
    
    def activation_backward(self,z,act_index):
        if(self.act[act_index]=="sigmoid"):
            return self.sigmoid_backward(z)
        elif (self.act[act_index]=="softplus"):
            return self.softplus_backward(z)
        elif (self.act[act_index]=="tanh"):
            return self.tanh_backward(z)
        elif (self.act[act_index]=="relu"):
            return self.relu_backward(z)
        else:
            print("error invalid activation")
    
    def feed_forward(self, input_arr,dropout=0):
#         print(input_arr.shape)
        p=1-dropout
        
        
        self.inps = np.ones((input_arr.shape[0],self.no_of_inputs+1))
        self.inps[:,1:self.no_of_inputs+1]=input_arr
        self.layers=[]
        self.layers.append(self.inps)
        u1 = np.random.binomial(1, p, size=self.layers[-1].shape) / p
        self.layers[-1] = self.layers[-1]*u1
        layer0nodes=np.matmul(self.layers[-1],self.weights[0])
        layer0nodes=self.activation_forward(layer0nodes,0)
        layer0nodes_bias=np.ones((layer0nodes.shape[0],layer0nodes.shape[1]+1))
        layer0nodes_bias[:,1:layer0nodes.shape[1]+1]=layer0nodes
        #for all the internal layer nodes
        self.layers.append(layer0nodes_bias)
        u1 = np.random.binomial(1, p, size=self.layers[-1].shape) / p
        self.layers[-1] = self.layers[-1]*u1
        for i in range(1,self.no_of_hidden_layers):
            layerinodes=np.matmul(self.layers[-1],self.weights[i])
            layerinodes=self.activation_forward(layerinodes,i)
            layerinodes_bias=np.ones((layerinodes.shape[0],layerinodes.shape[1]+1))
            layerinodes_bias[:,1:layerinodes.shape[1]+1]=layerinodes
            self.layers.append(np.copy(layerinodes_bias))
            u1 = np.random.binomial(1, p, size=self.layers[-1].shape) / p
            self.layers[-1] = self.layers[-1]*u1
            
        #for the last layer
        layer1nodes=np.matmul(self.layers[-1],self.weights[-1])
        layer1nodes=self.activation_forward(layer1nodes,-1)
        self.layers.append(layer1nodes)
#         print(len(self.layers))
        return self.layers[-1]
        
    def get_layers(self):
        return self.layers
    
    #can use with sigmoid
    def compute_cross_entropy_err(self,y_hat,y):
        #cross entropy, expects one hot encoded y label
        loss=0
        #ylog(y_hat)+(1-y)log(1-y_hat)
        loss_matrix=np.multiply(y,np.log(y_hat))+np.multiply(1-y,np.log(1-y_hat))
        loss_matrix=np.nan_to_num(loss_matrix)
        #computing sum of squares of weights
        sum_square=0
        for i in range(0,len(self.weights)):
            sum_square=sum_square+np.sum((np.array(self.weights[i]))**2)
        
        loss=-1*np.sum(loss_matrix)*(1/y.shape[0])+self.regu*(1/y.shape[0])*sum_square
        return loss
    
    def compute_gradients(self,y_hat,y):
        #dE/d(sigma)
        self.gradients=[]
        dE_dsigmaL=(-1)*(np.multiply(y,1/y_hat)-np.multiply(1-y,1/(1-y_hat)))
#         dE_dsigmaL=np.multiply(dE_dsigmaL_init,y)-np.multiply(dE_dsigmaL_init,1-y)
        dE_dsigmaL=np.nan_to_num(dE_dsigmaL)
        dE_dsigmaL_dsumL=np.multiply(dE_dsigmaL,self.activation_backward(self.layers[-1],-1))
        dE_dsigmaL_dsumL=np.nan_to_num(dE_dsigmaL_dsumL)
        #average
        #gradient for the last layer
        #update weight with this gradient
        dE_dsigmaL_dsumL_dw=np.matmul(self.layers[-2].T,dE_dsigmaL_dsumL)*(1/self.layers[-2].shape[0])
        self.gradients.insert(0,dE_dsigmaL_dsumL_dw)
        
        for i in range(self.no_of_hidden_layers,0,-1):
            #weights without bias
            weights_wo_bias=self.weights[i][1:,:]
            dE_dsigmaL_dsumL_dsigmal=np.matmul(dE_dsigmaL_dsumL,weights_wo_bias.T)
            layer_wo_bias=self.layers[i][:,1:]
            dE_dsigmaL_dsumL=np.multiply(dE_dsigmaL_dsumL_dsigmal,self.activation_backward(layer_wo_bias,i-1))
            dE_dsigmaL_dsumL=np.nan_to_num(dE_dsigmaL_dsumL)
            dE_dsigmal_dsuml_dw=np.matmul(self.layers[i-1].T,dE_dsigmaL_dsumL)*(1/self.layers[i-1].shape[0])
            self.gradients.insert(0,np.copy(dE_dsigmal_dsuml_dw))
        
        return self.gradients
    
    def update_weights(self, gradients,learning_rate=0.001):
        self.lr=learning_rate
        for i in range(0,len(self.weights)):
            self.weights[i]=np.copy(self.weights[i])-gradients[i]*learning_rate-self.regu*learning_rate*np.copy(self.weights[i])
    
    #dont use predict for now
    def predict(self,input_arr):
        inps = np.ones((input_arr.shape[0],self.no_of_inputs+1))
        inps[:,1:self.no_of_inputs+1]=input_arr
        self.layers=[]
#         self.layers.append(self.inps)
        layer0nodes=np.matmul(self.inps,self.weights[0])
        layer0nodes=self.sigmoid_forward(layer0nodes)
        layer0nodes_bias=np.ones((layer0nodes.shape[0],layer0nodes.shape[1]+1))
        layer0nodes_bias[:,1:layer0nodes.shape[1]+1]=layer0nodes
        #output layer nodes
#         self.layers.append(layer0nodes_bias)
        layer1nodes=np.matmul(layer0nodes_bias,self.weights[1])
        layer1nodes=self.sigmoid_forward(layer1nodes)
#         self.layers.append(layer1nodes)
#         print(len(self.layers))
        return layer1nodes
        
        
               

In [16]:
train_data.shape

network1=nn(no_of_inputs=24,no_of_outputs=3,HUs=[100],regu=0,no_of_hidden_layers=1, activation=["tanh","sigmoid"])
train_data.shape[1]

25

In [17]:
input_data=train_data.values
input_x=input_data[:,:-1]
y_label=input_data[:,-1]
y_encod=onehotencoding(y_label)
val_input=val_data.values
val_x=val_input[:,:-1]
y_val_label=val_input[:,-1]
y_val_encod=onehotencoding(y_val_label)
# y_encod=y_encod[[0],:]
# input_x=input_x[[0],:]

In [18]:
y_encod;


In [19]:
def train(input_x,y_encod,no_of_epochs=10000,batchSize=100):
    print(input_x.shape)
    print(y_encod.shape)
    no_of_batches=int(input_x.shape[0]/batchSize)
    lr=0.01
    for i in range(0,no_of_epochs):
        loss=0
        for batch_no in range(0,no_of_batches):
            idx = np.random.randint(input_x.shape[0], size=batchSize)
            input_x_train=input_x[idx,:]
            y_encod_train=y_encod[idx,:]
            y_pred=network1.feed_forward(input_x_train,dropout=0.5)

            loss=loss+network1.compute_cross_entropy_err(y_pred,y_encod_train)
        #         print(y_pred)
            grads=network1.compute_gradients(y_pred,y_encod_train)
        #     if(i%1000==0):
        #         print(grads)
        #         print(network1.get_weights())
            network1.update_weights(grads,learning_rate= lr)
        if i%50==0:
            loss=loss*1/no_of_batches
            print("train loss: "+ str(loss))
            
            #checking val loss
            y_pred_val=network1.feed_forward(val_x)
            val_loss=network1.compute_cross_entropy_err(y_pred_val,y_val_encod)
            print("val loss: "+ str(val_loss))
        if (i+1)%1000==0:
            lr=lr*0.5

In [20]:
train(input_x,y_encod,no_of_epochs=10000)

(1800, 24)
(1800, 3)
train loss: 2.0734420904719637
val loss: 2.0584255618773457
train loss: 1.5140111728670278
val loss: 1.5020817557835613
train loss: 1.4574987279066134
val loss: 1.4227334059935641
train loss: 1.442188650206538
val loss: 1.391892825946885
train loss: 1.4812457613432395
val loss: 1.3728256228936357
train loss: 1.4586795157052903
val loss: 1.3710421102244157
train loss: 1.4063561190681915
val loss: 1.367036237735373
train loss: 1.4291534373106092
val loss: 1.3663684927960775
train loss: 1.472606354026049
val loss: 1.3582201492010058
train loss: 1.355411725776159
val loss: 1.3586346799326157
train loss: 1.3957341356687898
val loss: 1.3588414270866396
train loss: 1.450246203555632
val loss: 1.3487249068692269
train loss: 1.427918544594484
val loss: 1.349156008388359
train loss: 1.3602592059450962
val loss: 1.3471608928070407
train loss: 1.4201154180508193
val loss: 1.349424808533767
train loss: 1.4199296419910628
val loss: 1.3462665383929369
train loss: 1.36978373713002

KeyboardInterrupt: 

In [ ]:
weights=network1.get_weights()
np.asarray(weights)

In [11]:
def trainAccuracy(input_x,y_encod):
    y_pred=network1.feed_forward(input_x)
    y_pred_encod=(y_pred == y_pred.max(axis=1)[:,None]).astype(int)
    print(y_pred_encod.shape)
#     print(np.argmax(y_pred_encod,axis=1).shape)
    correct_pred_encod=np.multiply(y_pred_encod,y_encod)
    accuracy=np.sum(correct_pred_encod)/input_x.shape[0]
    print(accuracy)
    
trainAccuracy(input_x,y_encod)

(28800, 3)
0.7351388888888889


In [12]:
def preprocessData_test(df):
    df['post_day'] = df['post_day'].factorize(sort=True)[0]
    df['basetime_day'] = df['basetime_day'].factorize(sort=True)[0]
#     df=df.head(5)
    df_norm=(df-train_mean)/train_std
    df_norm.fillna(0,inplace=True)
    return df_norm

In [14]:
df_test = pd.read_csv('./../data/test.csv')
test_data=preprocessData_test(df_test)
input_data=test_data.values

In [15]:
def testOutput(input_x_test):
    y_pred=network1.feed_forward(input_x_test)
    y_pred_arr=np.argmax(y_pred,axis=1)
    y_pred_arr=y_pred_arr+1
    file=open("submission.csv","w")
    file.write("Id,predicted_class\n")
    print(len(y_pred_arr))
    for i in range(0,len(y_pred_arr)):
        file.write(str(i+1)+","+str(y_pred_arr[i])+"\n")
    file.close()

testOutput(input_data)

10000


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: overflow encountered in exp


In [20]:
print(input_x.shape)
input_x_part=input_x
print(input_x_part.shape)
y_pred=network1.feed_forward(input_x_part)
print(y_pred)
weights=network1.get_weights()
print(weights)
layers=network1.get_layers()
print(layers)


(5, 2)
(5, 2)
[[0.49765536 0.50368654 0.49381384]
 [0.49764957 0.50367189 0.49383668]
 [0.49764852 0.50367562 0.4938206 ]
 [0.49765452 0.5036914  0.49379497]
 [0.49765335 0.50368932 0.4937968 ]]
[array([[ 0.01624345, -0.00611756, -0.00528172],
       [-0.01072969,  0.00865408, -0.02301539],
       [ 0.01744812, -0.00761207,  0.00319039]]), array([[-0.0024937 ,  0.01462108, -0.02060141],
       [-0.00322417, -0.00384054,  0.01133769],
       [-0.01099891, -0.00172428, -0.00877858],
       [ 0.00042214,  0.00582815, -0.01100619]])]
[array([[ 1.        , -0.60865379,  0.23565675],
       [ 1.        ,  1.18539118,  1.39296744],
       [ 1.        ,  0.98290729,  0.32057733],
       [ 1.        , -0.92479789, -1.00327539],
       [ 1.        , -0.63484679, -0.94592614]]), array([[1.        , 0.50672107, 0.49670536, 0.50236961],
       [1.        , 0.50695686, 0.49838439, 0.4929705 ],
       [1.        , 0.50282263, 0.49998708, 0.49328017],
       [1.        , 0.50216523, 0.49837905, 0.5032

In [33]:
dE_dsigmaL=(-1)*(np.multiply(y_encod,1/y_pred)+np.multiply(1-y_encod,1/(1-y_pred)))
print(dE_dsigmaL)


[[-1.99066523 -1.98536177 -1.97555777]
 [-1.99064225 -2.01479624 -2.02496096]
 [-1.99063813 -2.01481138 -2.02502692]
 [-2.00942613 -2.01487543 -1.97548412]
 [-1.99065726 -2.01486698 -2.0251245 ]]


In [34]:
dE_dsigmaL=np.multiply(dE_dsigmaL,y_encod)
print(dE_dsigmaL)
dE_dsigmaL_dsumL=np.multiply(dE_dsigmaL,network1.sigmoid_backward(layers[-1]))
print(dE_dsigmaL_dsumL)
print(network1.sigmoid_backward(layers[-1]))
print(weights[1].shape)

[[-0.         -1.98536177 -0.        ]
 [-0.         -0.         -2.02496096]
 [-0.         -0.         -2.02502692]
 [-2.00942613 -0.         -0.        ]
 [-0.         -0.         -2.0251245 ]]
[[-0.         -0.49631346 -0.        ]
 [-0.         -0.         -0.50616332]
 [-0.         -0.         -0.5061794 ]
 [-0.50234548 -0.         -0.        ]
 [-0.         -0.         -0.5062032 ]]
[[0.2499945  0.24998641 0.24996173]
 [0.24999448 0.24998652 0.24996201]
 [0.24999447 0.24998649 0.24996181]
 [0.2499945  0.24998637 0.2499615 ]
 [0.24999449 0.24998639 0.24996152]]
(4, 3)


In [35]:
print(layers[-2])
print(layers[-2].T)
print(dE_dsigmaL_dsumL)
dE_dsigmaL_dsumL_dw=np.matmul(layers[-2].T,dE_dsigmaL_dsumL)*(1/layers[-2].shape[0])
print(dE_dsigmaL_dsumL_dw)
#update weight with this gradient
print(dE_dsigmaL_dsumL.shape)
#weights without bias


[[1.         0.50672107 0.49670536 0.50236961]
 [1.         0.50695686 0.49838439 0.4929705 ]
 [1.         0.50282263 0.49998708 0.49328017]
 [1.         0.50216523 0.49837905 0.50320046]
 [1.         0.50163763 0.49889722 0.50157791]]
[[1.         1.         1.         1.         1.        ]
 [0.50672107 0.50695686 0.50282263 0.50216523 0.50163763]
 [0.49670536 0.49838439 0.49998708 0.49837905 0.49889722]
 [0.50236961 0.4929705  0.49328017 0.50320046 0.50157791]]
[[-0.         -0.49631346 -0.        ]
 [-0.         -0.         -0.50616332]
 [-0.         -0.         -0.5061794 ]
 [-0.50234548 -0.         -0.        ]
 [-0.         -0.         -0.5062032 ]]
[[-0.1004691  -0.09926269 -0.30370918]
 [-0.05045209 -0.0502985  -0.1530104 ]
 [-0.05007169 -0.04930431 -0.15157809]
 [-0.0505561  -0.04986656 -0.15062244]]
(5, 3)


In [32]:
print(dE_dsigmaL_dsumL)
weights_wo_bias=weights[1][1:,:]
print(weights_wo_bias.T)
dE_dsigmaL_dsumL_dsigmal=np.matmul(dE_dsigmaL_dsumL,weights_wo_bias.T)
print(dE_dsigmaL_dsumL_dsigmal)
print(dE_dsigmaL_dsumL_dsigmal.shape)
print(network1.sigmoid_backward(layers[-2]).shape)

[[-0.         -0.49631346 -0.        ]
 [-0.         -0.         -0.49383668]
 [-0.         -0.         -0.4938206 ]
 [-0.49765452 -0.         -0.        ]
 [-0.         -0.         -0.4937968 ]]
[[-0.00322417 -0.01099891  0.00042214]
 [-0.00384054 -0.00172428  0.00582815]
 [ 0.01133769 -0.00877858 -0.01100619]]
[[ 0.00190611  0.00085578 -0.00289259]
 [-0.00559897  0.00433519  0.00543526]
 [-0.00559879  0.00433505  0.00543508]
 [ 0.00160452  0.00547366 -0.00021008]
 [-0.00559852  0.00433484  0.00543482]]
(5, 3)
(5, 4)


In [10]:
dE_dsigmaL=(-1)*(np.multiply(y_encod,1/y_pred)+np.multiply(1-y_encod,1/(1-y_pred)))
print(dE_dsigmaL)

dE_dsigmaL_dsumL=np.multiply(dE_dsigmaL,network1.sigmoid_backward(layers[-1]))
print(dE_dsigmaL_dsumL)
print(layers[1])
print(weights[1].shape)
#average
dE_dsigmaL_dsumL_dw=np.matmul(layers[-2].T,dE_dsigmaL_dsumL)*(1/layers[-2].shape[0])
print(dE_dsigmaL_dsumL_dw)
#update weight with this gradient
print(dE_dsigmaL_dsumL.shape)
#weights without bias
weights_wo_bias=weights[1][1:,:]
dE_dsigmaL_dsumL_dsigmal=np.matmul(dE_dsigmaL_dsumL,weights_wo_bias.T)
print(dE_dsigmaL_dsumL_dsigmal)
print(dE_dsigmaL_dsumL_dsigmal.shape)
print(network1.sigmoid_backward(layers[-2]).shape)
#layer without bias
print(layers[-2])
layer_wo_bias=layers[-2][:,1:]
dE_dsigmal_dsuml=np.multiply(dE_dsigmaL_dsumL_dsigmal,network1.sigmoid_backward(layer_wo_bias))
print(dE_dsigmal_dsuml)
print(layers[0].shape)
print(dE_dsigmal_dsuml.shape)
dE_dsigmal_dsuml_dw=np.matmul(layers[0].T,dE_dsigmal_dsuml)*(1/layers[0].shape[0])
print(dE_dsigmal_dsuml_dw)
print(weights[0])
# dE_dsigmaL_dsumL_dw_mean=np.mean(dE_dsigmaL_dsumL_dw,axis=0)


[[-2.01468662 -2.0021407  -1.99843484]
 [-2.0147238  -1.99790979 -2.00170064]]
[[-0.50364489 -0.5005346  -0.4996084 ]
 [-0.50365405 -0.4994769  -0.5004248 ]]
[[1.         0.50537419 0.49874734]
 [1.         0.50550913 0.48727871]]
(3, 3)
[[-0.50364947 -0.50000575 -0.5000166 ]
 [-0.25456542 -0.2527237  -0.25272925]
 [-0.24830572 -0.24651238 -0.24651236]]
(2, 3)
[[ 0.00423022 -0.00211824]
 [ 0.00426253 -0.00213153]]
(2, 2)
(2, 3)
[[1.         0.50537419 0.49874734]
 [1.         0.50550913 0.48727871]]
[[ 0.00105743 -0.00052956]
 [ 0.0010655  -0.00053254]]
(2, 3)
(2, 2)
[[ 0.00106147 -0.00053105]
 [ 0.00030971 -0.00015447]
 [ 0.0008667  -0.0004333 ]]
[[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]
 [ 0.00865408 -0.02301539]]


[0.73105858 0.73105858 0.73105858 0.73105858]


In [36]:
range(10)

range(0, 10)

In [45]:
y_label

array([2., 3., 3., 1., 3.])

In [26]:
a=np.array([])
a=np.append(a,2)
a=np.append(a,3)
a.shape[0]
a[-1:]

array([3.])

In [24]:
for i in range(10,1,-1):
    print(i)

10
9
8
7
6
5
4
3
2


In [21]:
def a(b=[2,3]):
    c=b[0]
    print(c)

TypeError: 'builtin_function_or_method' object is not subscriptable

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]])